In [20]:
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
from datetime import timedelta

In [2]:
# Load data and sort them by time
df = pd.read_csv("./Data/traffic.csv")
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.sort_values(by=['DateTime'], inplace=True)

In [3]:
# Encode Junction as a numerical feature
junction_encoder = LabelEncoder()
df['Junction_encoded'] = junction_encoder.fit_transform(df['Junction'])

In [4]:
# Extract time-based features for better pattern recognization
df['Hour'] = df['DateTime'].dt.hour
df['DayOfWeek'] = df['DateTime'].dt.dayofweek
df['Month'] = df['DateTime'].dt.month
df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)

In [5]:
# Normalize data

scaler_dict = {}
numerical_features = ['Vehicles', 'Hour', 'DayOfWeek', 'Month', 'Junction_encoded']
for feature in numerical_features:
    scaler = MinMaxScaler()
    df[f'{feature}_normalized'] = scaler.fit_transform(df[[feature]])
    scaler_dict[feature] = scaler

In [ ]:
# LSTM model with multi-step prediction
class JunctionTrafficLSTM(nn.Module):
    def __init__(self, input_size=5, hidden_size=32, num_layers=5, output_size=1):
        super(JunctionTrafficLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
    
    def forward(self, x, future_steps=1, teacher_forcing_ratio=0.5):
        batch_size = x.size(0)
        outputs = []
        lstm_out, (h, c) = self.lstm(x)
        decoder_input = x[:, -1:, :]
        
        for i in range(future_steps):
            # Run LSTM step
            out, (h, c) = self.lstm(decoder_input, (h, c))
            
            # Process through
            features = self.dropout(out[:, -1])
            features = self.relu(self.fc1(features))
            features = self.relu(self.fc2(features))
            prediction = self.fc3(features)
            
            outputs.append(prediction)
        
            if self.training and torch.rand(1).item() < teacher_forcing_ratio:
                decoder_input = x[:, i+1:i+2, :] if i+1 < x.size(1) else decoder_input
            else:
                next_input = decoder_input.clone()
                next_input[:, 0, 0] = prediction.squeeze()
                decoder_input = next_input
        
        return torch.stack(outputs, dim=1)

In [ ]:
def create_multistep_sequences(data, input_seq_length, prediction_length):
    feature_cols = [col for col in data.columns if '_normalized' in col]
    sequences = []
    targets = []
    
    for junction in data['Junction'].unique():
        junction_data = data[data['Junction'] == junction].copy()
        
        for i in range(len(junction_data) - input_seq_length - prediction_length):
            # Input
            seq = junction_data[feature_cols].iloc[i:i+input_seq_length].values
            
            # Target
            target = junction_data['Vehicles_normalized'].iloc[i+input_seq_length:i+input_seq_length+prediction_length].values
            
            sequences.append(seq)
            targets.append(target)
    
    return np.array(sequences), np.array(targets)

In [ ]:
def train_multistep_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=10, prediction_length=24):
    best_val_loss = float('inf')
    patience_counter = 0
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            
            # Generate predictions
            outputs = model(batch_X, future_steps=prediction_length)
            
            # Ensure outputs and batch_y have the same shape
            outputs = outputs.squeeze()
            if len(outputs.shape) == 1:
                outputs = outputs.unsqueeze(0)
            if len(batch_y.shape) == 1:
                batch_y = batch_y.unsqueeze(0)

            loss = criterion(outputs, batch_y)

            if outputs.size(1) > 1:
                output_diff = outputs[:, 1:] - outputs[:, :-1]
                target_diff = batch_y[:, 1:] - batch_y[:, :-1]
                
                min_length = min(output_diff.size(1), target_diff.size(1))
                consistency_loss = criterion(
                    output_diff[:, :min_length],
                    target_diff[:, :min_length]
                )
                loss = loss + 0.1 * consistency_loss
            
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X, future_steps=prediction_length)
                
                outputs = outputs.squeeze()
                if len(outputs.shape) == 1:
                    outputs = outputs.unsqueeze(0)
                if len(batch_y.shape) == 1:
                    batch_y = batch_y.unsqueeze(0)
                    
                val_loss += criterion(outputs, batch_y).item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

In [9]:
def accuracy(model, test_loader, threshold, prediction_length=24):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X, future_steps=prediction_length)
            difference = torch.abs(outputs.squeeze() - batch_y)
            correct += (difference <= threshold).sum().item()
            total += batch_y.numel()
    
    accuracy_percentage = (correct / total) * 100
    print(f"Test Accuracy: {accuracy_percentage:.2f}%")
    return accuracy_percentage

In [ ]:
def predict_future_traffic(model, latest_time, num_hours, scalers, seq_length=10):
    model.eval()
    closest_time_idx = (df['DateTime'] - latest_time).abs().argmin()
    print((df['DateTime'] - latest_time).abs().min())

    feature_cols = [col for col in df.columns if '_normalized' in col]
    input_seq = df.iloc[closest_time_idx:closest_time_idx+seq_length][feature_cols].values
    input_seq = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).to(device)

    # Make prediction
    with torch.no_grad():
        predictions = model(input_seq, future_steps=num_hours)
        predictions = predictions.squeeze().cpu().numpy()

    # Inverse transform 
    predictions = scalers['Vehicles'].inverse_transform(predictions.reshape(-1, 1)).flatten()

    return predictions

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seq_length = 10
prediction_length = 24  # 24 hours ahead

In [ ]:
X, y = create_multistep_sequences(df, seq_length, prediction_length)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

val_size = int(0.1 * len(X_train))
X_val = X_train[-val_size:]
y_val = y_train[-val_size:]
X_train = X_train[:-val_size]
y_train = y_train[:-val_size]

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

In [13]:
model = JunctionTrafficLSTM().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
train_multistep_model(model, train_loader, val_loader, criterion, optimizer, 
                     num_epochs=50, patience=10, prediction_length=prediction_length)

Epoch 1, Train Loss: 0.0052, Val Loss: 0.0038
Epoch 2, Train Loss: 0.0019, Val Loss: 0.0025
Epoch 3, Train Loss: 0.0013, Val Loss: 0.0023
Epoch 4, Train Loss: 0.0011, Val Loss: 0.0024
Epoch 5, Train Loss: 0.0010, Val Loss: 0.0024
Epoch 6, Train Loss: 0.0010, Val Loss: 0.0022
Epoch 7, Train Loss: 0.0009, Val Loss: 0.0021
Epoch 8, Train Loss: 0.0009, Val Loss: 0.0021
Epoch 9, Train Loss: 0.0008, Val Loss: 0.0021
Epoch 10, Train Loss: 0.0008, Val Loss: 0.0020
Epoch 11, Train Loss: 0.0008, Val Loss: 0.0023
Epoch 12, Train Loss: 0.0007, Val Loss: 0.0021
Epoch 13, Train Loss: 0.0007, Val Loss: 0.0022
Epoch 14, Train Loss: 0.0007, Val Loss: 0.0020
Epoch 15, Train Loss: 0.0006, Val Loss: 0.0020
Epoch 16, Train Loss: 0.0006, Val Loss: 0.0021
Epoch 17, Train Loss: 0.0006, Val Loss: 0.0020
Epoch 18, Train Loss: 0.0006, Val Loss: 0.0020
Epoch 19, Train Loss: 0.0006, Val Loss: 0.0020
Epoch 20, Train Loss: 0.0006, Val Loss: 0.0019
Epoch 21, Train Loss: 0.0006, Val Loss: 0.0020
Epoch 22, Train Loss: 

In [ ]:
latest_time = pd.to_datetime("2015-11-03 07:00:00")
num_hours = 24

model.load_state_dict(torch.load('best_model.pth'))
predictions = predict_future_traffic(model, latest_time, num_hours, scaler_dict)

# Create prediction DataFrame
predicted_times = [latest_time + timedelta(hours=i) for i in range(num_hours)]
predicted_traffic_df = pd.DataFrame({
    'Predicted Time': predicted_times,
    'Predicted Traffic (Vehicles)': predictions
})

print(predicted_traffic_df)
predicted_traffic_df.to_csv('predicted_traffic.csv', index=False)

0 days 00:00:00
        Predicted Time  Predicted Traffic (Vehicles)
0  2015-11-03 07:00:00                     14.365445
1  2015-11-03 08:00:00                     13.884452
2  2015-11-03 09:00:00                     13.273060
3  2015-11-03 10:00:00                     12.776850
4  2015-11-03 11:00:00                     12.468714
5  2015-11-03 12:00:00                     12.187276
6  2015-11-03 13:00:00                     11.871385
7  2015-11-03 14:00:00                     11.457310
8  2015-11-03 15:00:00                     10.917954
9  2015-11-03 16:00:00                     10.190218
10 2015-11-03 17:00:00                      9.318969
11 2015-11-03 18:00:00                      8.423333
12 2015-11-03 19:00:00                      7.703674
13 2015-11-03 20:00:00                      7.228572
14 2015-11-03 21:00:00                      6.988348
15 2015-11-03 22:00:00                      6.991938
16 2015-11-03 23:00:00                      7.277221
17 2015-11-04 00:00:00        

In [19]:
threshold = 0.03
uncertainty = (scaler_dict['Vehicles'].inverse_transform([[threshold]]) - 
              scaler_dict['Vehicles'].inverse_transform([[0]]))[0][0]
print(f"\nAccuracy Evaluation:")
print(f"Threshold is {threshold}, which corresponds to {int(uncertainty)} car(s) uncertainty")
print("Predicted values within this uncertainty are considered correct in the calculation of accuracy")
accuracy(model, test_loader, threshold, prediction_length)


Accuracy Evaluation:
Threshold is 0.03, which corresponds to 5 car(s) uncertainty
Predicted values within this uncertainty are considered correct in the calculation of accuracy
Test Accuracy: 56.12%


56.11823139175437